In [397]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests
import asyncio
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [398]:
import re

def col_to_index(col):
    index = 0
    for c in col:
        index = index * 26 + (ord(c) - ord('A') + 1)
    return index - 1

def convert_range_to_indices(sheet_id, cell_range):
    """
    주어진 셀 범위를 rowIndex와 columnIndex로 변환합니다.
    
    :param sheet_id: 변환할 시트의 ID
    :param cell_range: 변환할 셀 범위 (예: "A2:B")
    :return: 변환된 범위 딕셔너리
    """
    
    
    # A1 표기법을 정규 표현식으로 분리
    match = re.match(r"([A-Z]+)([0-9]+):([A-Z]+)([0-9]+)", cell_range)
    if not match:
        raise ValueError("셀 범위 형식이 올바르지 않습니다. 예: 'A2:B10'")
    
    start_col, start_row, end_col, end_row = match.groups()
    
    start_row_index = int(start_row) - 1
    end_row_index = int(end_row)
    start_col_index = col_to_index(start_col)
    end_col_index = col_to_index(end_col) + 1
    
    return {
        'sheetId': sheet_id,
        'startRowIndex': start_row_index,
        'endRowIndex':end_row_index,
        'startColumnIndex': start_col_index,
        'endColumnIndex': end_col_index
    }

In [399]:
#셀 수정가능 경고표시로 셀 잠그기
def lock_cells_with_warning(cell_range, sheet_order_id):
    # cell_range = "A4:AG"
    range_value = convert_range_to_indices(sheet_order_id, cell_range)
    
    request = {
        "addProtectedRange": {
            "protectedRange": {
                "range": range_value,
                "requestingUserCanEdit": True,
                "description": "Warn on editing A4 to AG",
                "warningOnly": True
            }
        }
    }
    
    return request

In [400]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_to_function(formula, cell_range, sheet_order_id):
    range_name = f"{cell_range}"  # 지정된 시트의 특정 셀 범위    
    # spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    range_value = convert_range_to_indices(sheet_order_id, range_name);

    request = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {
                            "formulaValue": formula
                        }}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": range_value,
        }
    }
    return request

In [401]:
#체크표시 삽입
def add_checkbox_to_cell(cell_range, sheet_order_id):
    range_value = convert_range_to_indices(sheet_order_id, cell_range)
    
    request = {
        "updateCells": {
            "range": range_value,
            "rows": [{
                "values": [{
                    "userEnteredValue": {
                        "boolValue": False  # 기본값 FALSE
                    },
                    "dataValidation": {
                        "condition": {
                            "type": "BOOLEAN"
                        },
                        "showCustomUi": True
                    }
                }]
            }],
            "fields": "userEnteredValue,dataValidation"
        }
    }
    
    return request

In [402]:
# 특정 셀에 값을 넣는 함수
def apply_formula_to_text(value, cell_range, sheet_order_id):
    range_name = f"{cell_range}"  # 지정된 시트의 특정 셀 범위    
    
    range_value = convert_range_to_indices(sheet_order_id, range_name);

    if isinstance(value, int):
        user_entered_value = {"numberValue": value}
    elif isinstance(value, float):
        user_entered_value = {"numberValue": value}
    elif isinstance(value, bool):
        user_entered_value = {"boolValue": value}
    else:
        user_entered_value = {"stringValue": value}

    request = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": user_entered_value}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": range_value,
        }
    }
    return request

In [403]:
#범위 설정해주는 코드
def create_dropdown_request(sheet_id, range_name, dropdown_range, show_warning, show_arrow):
    range_value = convert_range_to_indices(sheet_id, range_name)
    
    request = {
        "repeatCell": {
            "range": range_value,
            "cell": {
                "dataValidation": {
                    "condition": {
                        "type": "ONE_OF_RANGE",
                        "values": [
                            {
                                "userEnteredValue": dropdown_range
                            }
                        ]
                    },
                    "strict": show_warning,
                    "showCustomUi": show_arrow
                }
            },
            "fields": "dataValidation"
        }
    }
    
    return request

#예시 
# range_str = "V4:V1000"
# dropdown_range = "='상품목록'!$K$5:$K"
# show_warning = True
# show_arrow = True

# request = create_dropdown_request(sheet_id, range_str, dropdown_range, show_warning, show_arrow)
    

In [404]:
def create_hide_columns_request(cell_range, sheet_id):
    range_value = convert_range_to_indices(sheet_id, cell_range)
    
    request = {
        "updateDimensionProperties": {
            "range": {
                "sheetId": sheet_id,
                "dimension": "COLUMNS",
                "startIndex": range_value['startColumnIndex'],
                "endIndex": range_value['endColumnIndex']
            },
            "properties": {
                "hiddenByUser": True
            },
            "fields": "hiddenByUser"
        }
    }
    return request


In [405]:
def create_clear_values_request(cell_range, sheet_id):
    range_value = convert_range_to_indices(sheet_id, cell_range)
    
    request = {
        "updateCells": {
            "range": range_value,
            "fields": "userEnteredValue"
        }
    }
    return request

In [406]:
def set_column_width(sheet_id, column, width):
    col_index = col_to_index(column)
    
    request = {
        "updateDimensionProperties": {
            "range": {
                "sheetId": sheet_id,
                "dimension": "COLUMNS",
                "startIndex": col_index,
                "endIndex": col_index + 1
            },
            "properties": {
                "pixelSize": width
            },
            "fields": "pixelSize"
        }
    }
    
    return request


In [407]:
def set_cell_format(cell_range, sheet_order_id, background_color=None, font_size=None, bold=None, border=None, horizontal_alignment=None, vertical_alignment=None):
    range_value = convert_range_to_indices(sheet_order_id, cell_range)
    
    user_entered_format = {}
    
    if background_color:
        user_entered_format["backgroundColor"] = {
            "red": background_color[0] / 255,
            "green": background_color[1] / 255,
            "blue": background_color[2] / 255
        }
    
    if font_size:
        user_entered_format["textFormat"] = user_entered_format.get("textFormat", {})
        user_entered_format["textFormat"]["fontSize"] = font_size
    
    if bold is not None:
        user_entered_format["textFormat"] = user_entered_format.get("textFormat", {})
        user_entered_format["textFormat"]["bold"] = bold

    if border:
        user_entered_format["borders"] = {}
        for position in ['top', 'right', 'bottom', 'left']:
            if position in border:
                user_entered_format["borders"][position] = {
                    "style": border[position]['style'],
                    "width": 1,
                    "color": {
                        "red": border[position]['color'][0] / 255,
                        "green": border[position]['color'][1] / 255,
                        "blue": border[position]['color'][2] / 255
                    }
                }
    
    if horizontal_alignment:
        user_entered_format["horizontalAlignment"] = horizontal_alignment
    
    if vertical_alignment:
        user_entered_format["verticalAlignment"] = vertical_alignment

    request = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredFormat": user_entered_format}
                    ]
                }
            ],
            "fields": "userEnteredFormat(backgroundColor,textFormat,borders,horizontalAlignment,verticalAlignment)",
            "range": range_value,
        }
    }
    return request


In [408]:
def merge_cells(cell_range, sheet_order_id):
    range_value = convert_range_to_indices(sheet_order_id, cell_range)

    request = {
        "mergeCells": {
            "range": range_value,
            "mergeType": "MERGE_ALL"
        }
    }
    return request


In [409]:
def copy_format_to_range(source_range, target_range, sheet_id):
    source_range_value = convert_range_to_indices(sheet_id, source_range)
    target_range_value = convert_range_to_indices(sheet_id, target_range)
    
    request = {
        "copyPaste": {
            "source": source_range_value,
            "destination": target_range_value,
            "pasteType": "PASTE_NORMAL"
        }
    }
    
    return request

In [410]:
def copy_format_to_range_서식(source_range, target_range, sheet_id):
    source_range_value = convert_range_to_indices(sheet_id, source_range)
    target_range_value = convert_range_to_indices(sheet_id, target_range)
    
    request = {
        "copyPaste": {
            "source": source_range_value,
            "destination": target_range_value,
            "pasteType": "PASTE_FORMATS"
        }
    }
    
    return request

In [411]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def get_sheet_id_by_title(service_sheets, destination_spreadsheet_id):
    spreadsheet = service_sheets.spreadsheets().get(spreadsheetId=destination_spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', '')
    sheet_order_id_발주서 = None
    sheet_order_id_발주서매핑모드 = None
    sheet_order_id_예판상품 = None
    sheet_order_id_누적발주 = None
    sheet_order_id_상품목록 = None
    
    for sheet in sheets:
        if sheet.get('properties', {}).get('title', '') == "발주서":
            sheet_order_id_발주서 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "발주서매핑모드":
            sheet_order_id_발주서매핑모드 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "누적발주":
            sheet_order_id_누적발주 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "상품목록":
            sheet_order_id_상품목록 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "예판상품":
            sheet_order_id_예판상품 = sheet.get('properties', {}).get('sheetId', '')

    return [sheet_order_id_발주서, sheet_order_id_발주서매핑모드, sheet_order_id_누적발주, sheet_order_id_상품목록, sheet_order_id_예판상품]


In [412]:
# @retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
# def get_protected_ranges(service, spreadsheet_id, sheet_id):
#     response = service.spreadsheets().get(
#         spreadsheetId=spreadsheet_id,
#         fields='sheets(protectedRanges)'
#     ).execute()
    
#     protected_ranges = []
#     for sheet in response.get('sheets', []):
#         if 'protectedRanges' in sheet:
#             for protected_range in sheet['protectedRanges']:
#                 if protected_range['range']['sheetId'] == sheet_id:
#                     protected_ranges.append(protected_range['protectedRangeId'])
#     return protected_ranges


@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def get_protected_ranges(service, spreadsheet_id, sheet_id):
    response = service.spreadsheets().get(
        spreadsheetId=spreadsheet_id,
        fields='sheets(protectedRanges)'
    ).execute()
    
    protected_ranges = []
    for sheet in response.get('sheets', []):
        if 'protectedRanges' in sheet:
            for protected_range in sheet['protectedRanges']:
                if 'range' in protected_range and protected_range['range'].get('sheetId') == sheet_id:
                    protected_ranges.append(protected_range['protectedRangeId'])
    return protected_ranges



def create_delete_protected_range_request(protected_range_id):
    return {
        "deleteProtectedRange": {
            "protectedRangeId": protected_range_id
        }
    }

In [413]:
# 드롭다운을 제거하는 요청 객체 생성 함수
def create_remove_dropdown_request(cell_range, sheet_id):
    range_value = convert_range_to_indices(sheet_id, cell_range)
    
    request = {
        "repeatCell": {
            "range": range_value,
            "cell": {
                "dataValidation": None
            },
            "fields": "dataValidation"
        }
    }
    return request

In [414]:
def create_unlock_all_requests(sheet_id):
    request = {
                "deleteProtectedRange": {
                    "range": {
                        "sheetId": sheet_id
                    }
                }

    }
    return request

In [415]:
def apply_protect_data(sheet_order_id, range_name):
    # 스프레드시트의 메타데이터에서 sheet_id 가져오기
    range_value = convert_range_to_indices(sheet_order_id, range_name);
    user_emails = ["daebong10x@gmail.com", "test-daebong@newdaebong.iam.gserviceaccount.com"]

        # 보호된 범위 설정
    request = {
        'addProtectedRange': {
            'protectedRange': {
                'range': range_value,
                'description': '수정을 원한다면 문의주세요',
                'editors': {
                    'users': user_emails,  # 이 셀을 편집할 수 있는 사용자
                }
            }
        }
    }

    return request

In [416]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_product_everyrow(range_name, sheet_order_id):
    range_value = convert_range_to_indices(sheet_order_id, range_name);
    requests = []
#     range_value['startColumnIndex'],
#     range_value['endColumnIndex']
    
    for i in range(6, 900):  # 6행부터 1010행까지
        formula = f'=iferror(VLOOKUP(V{i},$K:$L,2,0),"")'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex": range_value['startColumnIndex'],  # Column K (0-based index)
                    "endColumnIndex": range_value['endColumnIndex']
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [417]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_mapping_check_everyrow(range_name, sheet_order_id):
    range_value = convert_range_to_indices(sheet_order_id, range_name);
    requests = []
#     range_value['startColumnIndex'],
#     range_value['endColumnIndex']
    
    for i in range(6, 900):  # 6행부터 1010행까지
        formula = f'=IF(AND(ISBLANK(\'상품목록\'!T{i}), ISBLANK(\'상품목록\'!V{i}), ISBLANK(\'상품목록\'!W{i})), "", IF(OR(ISBLANK(\'상품목록\'!T{i}), ISBLANK(\'상품목록\'!V{i}), ISBLANK(\'상품목록\'!W{i})), "[매핑오류] 상품명 다시 확인해주세요.", "[정상매핑] 정상처리 되었습니다."))'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex": range_value['startColumnIndex'],  # Column K (0-based index)
                    "endColumnIndex": range_value['endColumnIndex']
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [418]:

# 특정 시트의 마지막 행을 구하는 함수
def get_last_row(service, spreadsheet_id, sheet_name):
    range_name = f'{sheet_name}!A:A'
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])
    
    last_row = len(values)
    return last_row

# 시트 ID와 마지막 행 번호를 가져오는 함수
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def get_sheet_id_and_last_row_by_title(service_sheets, destination_spreadsheet_id):
    spreadsheet = service_sheets.spreadsheets().get(spreadsheetId=destination_spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', [])
    
    sheet_info = {}
    
    for sheet in sheets:
        title = sheet.get('properties', {}).get('title', '')
        sheet_id = sheet.get('properties', {}).get('sheetId', '')
        
        if title:
            last_row = get_last_row(service_sheets, destination_spreadsheet_id, title)
            sheet_info[title] = {
                'sheet_id': sheet_id,
                'last_row': last_row
            }

    return sheet_info

In [419]:
def apply_dropdown_validation(dropdown_range, cell_range, sheet_order_id):
    range_value = convert_range_to_indices(sheet_order_id, cell_range);
    
    # 드롭다운 유효성 검사 설정
    request = {
                'repeatCell': {
                    'range': range_value,
                    'cell': {
                        'dataValidation': {
                            'condition': {
                                'type': 'ONE_OF_RANGE',
                                'values': [
                                    {
                                        'userEnteredValue': f"={dropdown_range}"
                                    }
                                ]
                            },
                            'showCustomUi': True, # 드롭다운 화살표 표시
                            'strict': False # 경고 표시
                        }
                    },
                    'fields': 'dataValidation'
                }
    }
    
    return request

In [420]:
def apply_single_value_dropdown(value, cell_range, sheet_id):
    range_value = convert_range_to_indices(sheet_id, cell_range)
    
    # 드롭다운 유효성 검사 설정
    request = {
        'repeatCell': {
            'range': range_value,
            'cell': {
                'dataValidation': {
                    'condition': {
                        'type': 'ONE_OF_LIST',
                        'values': [
                            {
                                'userEnteredValue': value
                            }
                        ]
                    },
                    # 'strict': True,  # 입력 거부
                    # 'showCustomUi': False  # 칩 형식
                     'showCustomUi': False,  # 화살표 대신 텍스트 칩 사용
                    'strict': True  # 입력 거부
                }
            },
            'fields': 'dataValidation'
        }
    }
    
    return request

In [421]:
#     ["21", "1U_HI5Th-1Yh9RBWiOGljs-NhXazxPl1HkbPQ9IZrKzA"], 
# ["335", "13bZWoE-oHVuPaPQXRYxIjDDAqmpvnqETYuP5S94Re08"], 
# ["442", "1E2i1IJRuPAoDmXum5vjrNo65mmaEjIyBKhENEH7VZEE"], 
# ["504", "1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw"], 
# ["516", "1VhHatV-asswfTDa9M13B8Orf2aDSmbdYWBzwrI4dqjM"], 
# ["523", "1y6sJA3EAAqaSLPgZMWYJjY5t2IZLrZGMBTXWI7sroIg"], 
# ["555", "1Pu9P4zFoDdFXK9ER_rZaEj2fa0FX36YEyCZIAV-YkEw"],
    # ['1', '1XYx2lJZAYpH2pdFiMPA-3QV9rLNtQ4S7F27d5msEEno', '최재혁'], ['3', '1G1qctmwr3pMKrhMHv1hMbvXrmmCI1TsJ5OLqKKg7U8c', '백민기'], 
    # ['20', '1D86pGcpOsFWp6IWb_KCGnNJerYcIiFNYzptaR2m21WI', '권오상(오에스인'], 
    # # ['21', '1U_HI5Th-1Yh9RBWiOGljs-NhXazxPl1HkbPQ9IZrKzA', '왕선균(오늘곳간)'], 
    # ['42', '1ziAp-yyjSgHesi8OKrzl9YxQYufx8Q9rUhUFaT5pWzI', '42'], ['47', '1WgI0QIOdj_NS7JdtIgDdYr-EAI5mTVnjQV3iVm5PP40', '47'], ['48', '1JScmTyIC8YKuRE867hRjiP2_eh4DAVmlJMw-zdhKw2s', '48랑랑이네'], ['50', '1MOpKL9N6ZhdqR6OBEq7AL2YekDCZeLs1VOJv35VphEw', '장진웅'], ['56', '1XmTgyrxTdK3hcHUm1YwYkoBom51v-y2jZTwC0A1Bn8U', '송용재'], ['57', '1ps2R_HXXHEuoKz7glv6LzlBZVyMNvfYNx23kQOxdnXE', '57'], ['60', '1erbDgg1807ZRZhRsoZJr6uYlGgLpT557dt81JxsYO-w', '60'], ['90', '1ZTgyd5YQjluAL8pLVKVqk-BqrOl6EgST47Gd8OtP_RA', '90'], ['92', '15DEt2XorXfKa-UlJtHsauu2b4dmS58RJ3fx1fS1zwkY', '92'], ['97', '1andpTzI55GWGRXV-JjaAtSIE3m4i7fDCHU2CNSk9TEU', '방성식'], ['98', '1ppOTzTz8_QMET4m2z0i6-aMg136hwfi1VeL2HCACyh4', '강대혁'], ['114', '1mZVlL4QwI2eHByQEpYs0dLYUQX4OcSa4tsdTJPR2VRI', '114'], ['121', '1ZEtoHfB5c0PJe2cE7TnP5Q5j8nLrc8hF7lPpdemH90A', '121'], ['130', '1MZN4QGtYktUqUXd6DFPxfyIustQ2BciBQdZd7-8jiOU', '보연주식회사'], ['138', '1Ync5OXAnvBk847u0dDiNkg-H11Ma3CKWzu036NCvPUg', '138'], ['143', '1LbXFDggoMRGJPUTgYp9I5BRvJZ3Fm20oQVL82ymRUl0', '143'], ['171', '1ZOX9E8BaYZ9IvVYZ1AeIQ7--PW_QYv7uHviHP-bqOyU', '우리과일맛있게맛'], ['178', '1hoPzaAfLCA3sV_WMrgNd_aQlIXEkBty4nwVcDZG1Pq0', '178무지개식탁'], ['180', '1WkE6NCdspuCQVIi2mGBXiA1GaFN05-l_9y9Iymu3Ln0', '180'], ['189', '1O1F3u9rP1w1zPCBwvgBhzOZxZdcHxgh3ZqbsRBBWI14', '안호현(마싯농)'], ['192', '1TSnCCP6nJ4PljZBsoqru23j8d53pfLJf5S0pK6tDAXM', '192_하이푸드'], ['195', '1q_g0-xOyXOm02BLASm7kyasalv6bWGO2RBF-Gbn6gGw', '다오소'], ['208', '1djE04qMRnEYYLwY0y4qEpHnBVWwOBxAimrzLgTuBe6k', '２０８'], ['209', '1GBPzy-GuxujWEGkH606Eb72-D8lo4yEKrZP7DrGzWFk', '209'], ['226', '1-J6V0nLHQlxLA8PbTiTXaGt08VlqGv5fo3kfaH35FJE', '김정재'], ['230', '1OxaNX2XUK5CTsqseH6Dnj_3Bhi1FNn_79YAAH1sOdhE', '230'], ['260', '1xD9hJnw7s8BNMuFGpC7uiO2YJsjNPRYQoObvNsxo4sE', '송현우'], ['276', '1V0EBc39eBX1ekf8DGFvCiS5SuVnTm6TehpOkhof7Jlo', '276_김도환(청년'], ['283', '1ldQveZcPwtieVcN-zlIzhXBxO3xnY7523gNNQpnPrfE', '283'], ['325', '14SbtLIs4ZPfiaJh-kRb-TE4RtOcO-gw4knuy3MolLX8', '이호연(컨프래'], ['328', '1wcx04SVtzkmlPl-PvqihBnRYHXnaNZ6EJykoHjME5hQ', '전혜린'], ['330', '1bwyuRqmCmNMr0-0qv_kglHL2olWFx-92MvbF-L8iRZY', '330'], 
    # # ['335', '13bZWoE-oHVuPaPQXRYxIjDDAqmpvnqETYuP5S94Re08', '홀핸드'], 
    # ['337', '10JQGUWjXCWbhmL7fq8cJfCmtBBIiDaOrqdjh1U1EI_U', '337_박정윤'], ['339', '1GCUBnyUr25uhB3HozLbiUQ5JngLhr7rfoNNR1cxAo58', '339슈거팜'], ['343', '1pd06-Meiiz1sntlIlMd49T6jS9Y6ZiXJU1gRs_hr_bE', '343'], 
    # ['353', '1jdRXB72CFDd35tj3Vmr7Ig2OXJfZX3x6wTLl0imas2M', '프리미엄찌니샵'], ['367', '1PXjuzLoiXYK9hLjmiSpsKMxnGXc09N_ZWxoaXPvMe-M', '367'], ['387', '1rNW5Kwv4M59HwihbR5FxiHq8MsvepJQsa1gRzrkmwME', '387'], ['406', '1fBNPp2YdC_xpb3W_4jGkZWRfV7l6_Vn96Vt44LQy8HE', '신윤식'], ['436', '1Ssu9s18Gug3HLRZNp3Hr_TddM2IOcPhlGE7ZjLRPsGo', '김홍곤(주윤)'], ['438', '1zWRhxaoNt8egvqUGmkRRf6KEFccJTDH-HleiKce9ZIM', '김윤성(오손도손)'], ['442', '1E2i1IJRuPAoDmXum5vjrNo65mmaEjIyBKhENEH7VZEE', '임경섭(프루딧)'], ['443', '1IOCubZrxV-dR1uNCFSgP3juhLIlMIMHYVaJACAMu1tY', '서영희(담다컴퍼'], ['461', '104ZWc3RoB0WjkITYQLyFBLvkZ5UywvIJZHoWIha9iTk', '우리동네남매마켓'], ['464', '1JG7u3zTmaH_WfKri2MvNGfUwGua3WAaCj-fXW6vSW2w', '조재훈(퍼플로드)'], ['468', '1dMLFlUP-oXYHnmY-Qm4hK0QUykH6sZyPdPza35HPWwU', '오정애'], ['470', '1oQvBGq3yKYDN9GxaK95bmVO99aVwDn9KGxJ_ceI2BFo', '최주연'], ['474', '1XRUFTiPVeZqLj9p1bw3t-qL-B-jy4Qteah6F1W216MA', '이우람'], ['475', '1ElTPyHmukcSKHDc4snsCcGfciEvOnngzXMB-3ccqdFo', '팔도장터'], ['485', '1vVGjqAgaOFovhHhrcpGkHmGktJug0f0zKgYIERT2FTA', '485정준희'], ['486', '1Uxgzhqwf-PLUE7OO9_Vzpkrr9i1N8PrVmvSpCjy6Pyo', '어썸파머스(주)'], ['489', '1RshPDxepseX4rzT6NxqCjVCDRu-I7zr8qUBgX3AxNQY', '이종철(수수마켓)'], ['496', '1xAcpRH5z51YF3dezUkBwc78vxWPfJOXquCN_4OkcuDM', '주식회사엠디어스'], ['497', '1bIOa6_1v7Jk3t2wAaPyo2G-hACN2iadRuNuVJbYMUVA', '497'], ['498', '1Bzna425shVhelODCpen2FUdiGu24KtR7jQUPNhaZkDY', '배민우'], ['500', '1asRBkSVowjMXIFWkeYZuqyTQmbpUhZi6ht3y66k4HE0', '함영민(과일파는'], ['503', '1np5z7X2XVUyTu1sOH_J0RREVPDNVK8CuN7CG8Jjf1K4', '주식회사와이앤제'], 
    # # ['504', '1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw', '504_팜브릿지'], 
    # ['506', '1HgYKDeGrRGxwg-j-yVqeZe6sJY1ip5mDJQ2lj4jLPt0', 'SH프레시푸드'], ['513', '1_eWdxS4Xdykn-GbZTTyOdH8f41GEvwURRcisG-PI9XU', '황소영'], ['514', '1Z0BJDjKsvPOrYXKD93NyRLUu--ecxpNue05n_UPLJjc', '주식회사호신호'], ['516', '1VhHatV-asswfTDa9M13B8Orf2aDSmbdYWBzwrI4dqjM', '김성헌'], ['517', '133IbYEp-aXQH-ECojwEEak6LKRgPKraS0yqTSvqNvWM', '김가람(과람아저'], ['523', '1y6sJA3EAAqaSLPgZMWYJjY5t2IZLrZGMBTXWI7sroIg', '김지영(동지코퍼'], ['525', '1zyM4XOmz1hRUwLM-2WXHh5BaJ-TMowTA0abn8cd7fe4', '525'], ['527', '1RZUWJpKa9b3CiBFNGkgUwHV2nJo1BE1gs9Uoe1XZmuM', '정민지'], ['537', '1kCgT5DWV1CcHTLJldd6BgiNU-Ara1zkX1TFKDMXiSms', '바이티존'], ['538', '1c3mQbrzmSfS19tAA7MWtVyP_DoT47KI3MPKhI24TUhE', '김규리,김경률'], ['541', '17sd8bF7sHRXpj8cEfTJ_44zhn8ALvSqJ26hkZlpU5ik', '그록정산금'], ['542', '1sClq5Zx710IIh5GhHzm6SdIgeZGmHVbmb8rNetSytJM', '조성현'], ['543', '1fmpcNUCK3wxx1M5TvCt1Cr00wtAaFwoUwPIFMnir5xw', '543_최혜정'], ['545', '1iC421AvucDQacamyihRfIP5WymejCEOC6Fd8t2vkre0', '최정훈(조니마켓)'], 
    # ['546', '1xUh--GSPTPLmMXXyHLiJ6DjyL3J9BG3py3SeSGJHkcw', '김혜연(어랑,과일'], ['547', '17A2-P3z_NA1BEGUXxRPEXHmpzRgKCmeA3hDuUWyQ3Xk', '이철호'], ['552', '1zt-VXJNilmSDYDzEhZqyV12Ern3WVu2-IivHWSJ4H1o', '가순성'], ['553', '1SCC7A90vXM5srUZpubCyc02Px3RsCpOeUjC8_Nl5g88', '김지인'], ['554', '1F1N6d2Co-E7LbNfnuj_xfTcJczBdUcVjQugp8sUi_3o', '싱그린청과'], ['555', '1Pu9P4zFoDdFXK9ER_rZaEj2fa0FX36YEyCZIAV-YkEw', '(주)마니쿡스'], ['559', '1PLYZbwJy_4EqiWoQw7lMAC6RGc9f6-BEN92yZmAWRZI', '박지환(와치맨컴'], ['562', '1jYeZFz3CNgXkBV3z1CsoD8k_e6xH-vZa3M0KboBD8rk', '우리DC마켓']
    # ['테스트', "1L3WeDB8dqwg7q_ubQdMVSYFisUGuAfQ2Txlb9V8excs"]

In [422]:
data_list = [
    ['20', '1D86pGcpOsFWp6IWb_KCGnNJerYcIiFNYzptaR2m21WI', '권오상(오에스인'], 
    ['21', '1U_HI5Th-1Yh9RBWiOGljs-NhXazxPl1HkbPQ9IZrKzA', '왕선균(오늘곳간)'], 
    ['42', '1ziAp-yyjSgHesi8OKrzl9YxQYufx8Q9rUhUFaT5pWzI', '42'], 
    ['47', '1WgI0QIOdj_NS7JdtIgDdYr-EAI5mTVnjQV3iVm5PP40', '47'], 
    ['48', '1JScmTyIC8YKuRE867hRjiP2_eh4DAVmlJMw-zdhKw2s', '48랑랑이네'], 
    ['50', '1MOpKL9N6ZhdqR6OBEq7AL2YekDCZeLs1VOJv35VphEw', '장진웅'], 
    ['56', '1XmTgyrxTdK3hcHUm1YwYkoBom51v-y2jZTwC0A1Bn8U', '송용재'], 
    ['57', '1ps2R_HXXHEuoKz7glv6LzlBZVyMNvfYNx23kQOxdnXE', '57'], ['60', '1erbDgg1807ZRZhRsoZJr6uYlGgLpT557dt81JxsYO-w', '60'], ['90', '1ZTgyd5YQjluAL8pLVKVqk-BqrOl6EgST47Gd8OtP_RA', '90'], ['92', '15DEt2XorXfKa-UlJtHsauu2b4dmS58RJ3fx1fS1zwkY', '92'], ['97', '1andpTzI55GWGRXV-JjaAtSIE3m4i7fDCHU2CNSk9TEU', '방성식'], ['98', '1ppOTzTz8_QMET4m2z0i6-aMg136hwfi1VeL2HCACyh4', '강대혁'], ['114', '1mZVlL4QwI2eHByQEpYs0dLYUQX4OcSa4tsdTJPR2VRI', '114'], ['121', '1ZEtoHfB5c0PJe2cE7TnP5Q5j8nLrc8hF7lPpdemH90A', '121'], ['130', '1MZN4QGtYktUqUXd6DFPxfyIustQ2BciBQdZd7-8jiOU', '보연주식회사'], ['138', '1Ync5OXAnvBk847u0dDiNkg-H11Ma3CKWzu036NCvPUg', '138'], ['143', '1LbXFDggoMRGJPUTgYp9I5BRvJZ3Fm20oQVL82ymRUl0', '143'], ['171', '1ZOX9E8BaYZ9IvVYZ1AeIQ7--PW_QYv7uHviHP-bqOyU', '우리과일맛있게맛'], ['178', '1hoPzaAfLCA3sV_WMrgNd_aQlIXEkBty4nwVcDZG1Pq0', '178무지개식탁'], ['180', '1WkE6NCdspuCQVIi2mGBXiA1GaFN05-l_9y9Iymu3Ln0', '180'], ['189', '1O1F3u9rP1w1zPCBwvgBhzOZxZdcHxgh3ZqbsRBBWI14', '안호현(마싯농)'], ['192', '1TSnCCP6nJ4PljZBsoqru23j8d53pfLJf5S0pK6tDAXM', '192_하이푸드'], ['195', '1q_g0-xOyXOm02BLASm7kyasalv6bWGO2RBF-Gbn6gGw', '다오소'], ['208', '1djE04qMRnEYYLwY0y4qEpHnBVWwOBxAimrzLgTuBe6k', '２０８'], ['209', '1GBPzy-GuxujWEGkH606Eb72-D8lo4yEKrZP7DrGzWFk', '209'], ['226', '1-J6V0nLHQlxLA8PbTiTXaGt08VlqGv5fo3kfaH35FJE', '김정재'], ['230', '1OxaNX2XUK5CTsqseH6Dnj_3Bhi1FNn_79YAAH1sOdhE', '230'], ['260', '1xD9hJnw7s8BNMuFGpC7uiO2YJsjNPRYQoObvNsxo4sE', '송현우'], ['276', '1V0EBc39eBX1ekf8DGFvCiS5SuVnTm6TehpOkhof7Jlo', '276_김도환(청년'], ['283', '1ldQveZcPwtieVcN-zlIzhXBxO3xnY7523gNNQpnPrfE', '283'], ['325', '14SbtLIs4ZPfiaJh-kRb-TE4RtOcO-gw4knuy3MolLX8', '이호연(컨프래'], ['328', '1wcx04SVtzkmlPl-PvqihBnRYHXnaNZ6EJykoHjME5hQ', '전혜린'], ['330', '1bwyuRqmCmNMr0-0qv_kglHL2olWFx-92MvbF-L8iRZY', '330'], ['335', '13bZWoE-oHVuPaPQXRYxIjDDAqmpvnqETYuP5S94Re08', ''], ['337', '10JQGUWjXCWbhmL7fq8cJfCmtBBIiDaOrqdjh1U1EI_U', '337_박정윤'], ['339', '1GCUBnyUr25uhB3HozLbiUQ5JngLhr7rfoNNR1cxAo58', '339슈거팜'], ['343', '1pd06-Meiiz1sntlIlMd49T6jS9Y6ZiXJU1gRs_hr_bE', '343'], ['353', '1jdRXB72CFDd35tj3Vmr7Ig2OXJfZX3x6wTLl0imas2M', '프리미엄찌니샵'], ['367', '1PXjuzLoiXYK9hLjmiSpsKMxnGXc09N_ZWxoaXPvMe-M', '367'], ['387', '1rNW5Kwv4M59HwihbR5FxiHq8MsvepJQsa1gRzrkmwME', '387'], ['406', '1fBNPp2YdC_xpb3W_4jGkZWRfV7l6_Vn96Vt44LQy8HE', '신윤식'], ['436', '1Ssu9s18Gug3HLRZNp3Hr_TddM2IOcPhlGE7ZjLRPsGo', '김홍곤(주윤)'], ['438', '1zWRhxaoNt8egvqUGmkRRf6KEFccJTDH-HleiKce9ZIM', '김윤성(오손도손)'], ['442', '1E2i1IJRuPAoDmXum5vjrNo65mmaEjIyBKhENEH7VZEE', '임경섭(프루딧)'], ['443', '1IOCubZrxV-dR1uNCFSgP3juhLIlMIMHYVaJACAMu1tY', '서영희(담다컴퍼'], ['461', '104ZWc3RoB0WjkITYQLyFBLvkZ5UywvIJZHoWIha9iTk', '우리동네남매마켓'], ['464', '1JG7u3zTmaH_WfKri2MvNGfUwGua3WAaCj-fXW6vSW2w', '조재훈(퍼플로드)'], ['468', '1dMLFlUP-oXYHnmY-Qm4hK0QUykH6sZyPdPza35HPWwU', '오정애'], ['470', '1oQvBGq3yKYDN9GxaK95bmVO99aVwDn9KGxJ_ceI2BFo', '최주연'], ['474', '1XRUFTiPVeZqLj9p1bw3t-qL-B-jy4Qteah6F1W216MA', '이우람'], ['475', '1ElTPyHmukcSKHDc4snsCcGfciEvOnngzXMB-3ccqdFo', '팔도장터'], ['485', '1vVGjqAgaOFovhHhrcpGkHmGktJug0f0zKgYIERT2FTA', '485정준희'], ['486', '1Uxgzhqwf-PLUE7OO9_Vzpkrr9i1N8PrVmvSpCjy6Pyo', '어썸파머스(주)'], ['489', '1RshPDxepseX4rzT6NxqCjVCDRu-I7zr8qUBgX3AxNQY', '이종철(수수마켓)'], ['496', '1xAcpRH5z51YF3dezUkBwc78vxWPfJOXquCN_4OkcuDM', '주식회사엠디어스'], ['497', '1bIOa6_1v7Jk3t2wAaPyo2G-hACN2iadRuNuVJbYMUVA', '497'], ['498', '1Bzna425shVhelODCpen2FUdiGu24KtR7jQUPNhaZkDY', '배민우'], ['500', '1asRBkSVowjMXIFWkeYZuqyTQmbpUhZi6ht3y66k4HE0', '함영민(과일파는'], ['503', '1np5z7X2XVUyTu1sOH_J0RREVPDNVK8CuN7CG8Jjf1K4', '주식회사와이앤제'], ['504', '1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw', '504_팜브릿지'], ['506', '1HgYKDeGrRGxwg-j-yVqeZe6sJY1ip5mDJQ2lj4jLPt0', 'SH프레시푸드'], ['513', '1_eWdxS4Xdykn-GbZTTyOdH8f41GEvwURRcisG-PI9XU', '황소영'], ['514', '1Z0BJDjKsvPOrYXKD93NyRLUu--ecxpNue05n_UPLJjc', '주식회사호신호'], ['516', '1VhHatV-asswfTDa9M13B8Orf2aDSmbdYWBzwrI4dqjM', '김성헌'], ['517', '133IbYEp-aXQH-ECojwEEak6LKRgPKraS0yqTSvqNvWM', '김가람(과람아저'], ['523', '1y6sJA3EAAqaSLPgZMWYJjY5t2IZLrZGMBTXWI7sroIg', '김지영(동지코퍼'], ['525', '1zyM4XOmz1hRUwLM-2WXHh5BaJ-TMowTA0abn8cd7fe4', '525'], ['527', '1RZUWJpKa9b3CiBFNGkgUwHV2nJo1BE1gs9Uoe1XZmuM', '정민지'], ['537', '1kCgT5DWV1CcHTLJldd6BgiNU-Ara1zkX1TFKDMXiSms', '바이티존'], ['538', '1c3mQbrzmSfS19tAA7MWtVyP_DoT47KI3MPKhI24TUhE', '김규리,김경률'], ['541', '17sd8bF7sHRXpj8cEfTJ_44zhn8ALvSqJ26hkZlpU5ik', '그록정산금'], ['542', '1sClq5Zx710IIh5GhHzm6SdIgeZGmHVbmb8rNetSytJM', '조성현'], ['543', '1fmpcNUCK3wxx1M5TvCt1Cr00wtAaFwoUwPIFMnir5xw', '543_최혜정'], ['545', '1iC421AvucDQacamyihRfIP5WymejCEOC6Fd8t2vkre0', '최정훈(조니마켓)'], ['546', '1xUh--GSPTPLmMXXyHLiJ6DjyL3J9BG3py3SeSGJHkcw', '김혜연(어랑,과일'], ['547', '17A2-P3z_NA1BEGUXxRPEXHmpzRgKCmeA3hDuUWyQ3Xk', '이철호'], ['552', '1zt-VXJNilmSDYDzEhZqyV12Ern3WVu2-IivHWSJ4H1o', '가순성'], ['553', '1SCC7A90vXM5srUZpubCyc02Px3RsCpOeUjC8_Nl5g88', '김지인'], ['554', '1F1N6d2Co-E7LbNfnuj_xfTcJczBdUcVjQugp8sUi_3o', '싱그린청과'], ['555', '1Pu9P4zFoDdFXK9ER_rZaEj2fa0FX36YEyCZIAV-YkEw', '(주)마니쿡스'], ['559', '1PLYZbwJy_4EqiWoQw7lMAC6RGc9f6-BEN92yZmAWRZI', '박지환(와치맨컴'], ['562', '1jYeZFz3CNgXkBV3z1CsoD8k_e6xH-vZa3M0KboBD8rk', '우리DC마켓']
]

In [423]:
excluded_list = ['20', '21', '42', '47', '48', '50', '56', '57', '60', '90', '92', '97', '98', '114', '121', '130', '138', '143', '171', '178', '180', '189', '192', '195', '208', '209', '226', '230', '260', '276', '283', '325', '328', '330', '335', '337', '339', '343', '353', '367', '387', '406', '436', '438', '442', '443', '461', '464', '468', '470', '474', '475', '485', '486', '489', '496', '497', '498', '500', '503', '504', '506', '513', '514', '516', '517', '523', '525', '527', '537', '538', '541', '542', '543', '545', '546', '547', '552', '553', '554', '555', '559', '562']
# list = []
# for i in range(20, 605):
#     if str(i) in excluded_list:
#             list[i] 
#             continue


In [424]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def batchWriteRequest(service, spreadsheet_id, all_requests):
    body = {
        'requests': all_requests
    }

    # batchUpdate 호출
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=body
    ).execute()
    
    return response


In [425]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
def get_values_from_range(service, spreadsheet_id, range_name):
    # Google Sheets API를 사용하여 특정 범위의 값 가져오기
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    # 값이 있는지 확인 후 반환
    values = result.get('values', [])

    return values

In [446]:
# excluded_titles = [21, 335, 442, 504, 516, 523, 555]
archive_spreadsheet_id = "1sAT1uc8EFval03w8sIubd8uDv8q-puYTB8KjEcjB75s"


if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    # user_data_list = get_values_from_range(service_sheets, archive_spreadsheet_id, "유저회원가입데이터!A389:K586") #회원가입 연동 테이블

    # print("user_data_list")
    # print(user_data_list)
    # new_data = [[item[1], item[-1]] for item in user_data_list]

    # print(new_data)


    new_data = [
        
    ]
    
    #0 ~ 5
    # 104에서 멈춤
    # 146에서 멈춤

    for data in new_data:
        # spreadsheet_title = data[0]
        spreadsheet_title = data[0]
        spreadsheet_id = data[1]
        print("spreadsheet_title")
        print(spreadsheet_title)
        if str(spreadsheet_title) in excluded_list:
            print("skip", spreadsheet_title)
            continue

        requests = []
        

        if not spreadsheet_id:
            continue



        background_color = (255, 229, 152)


        # data = await get_sheet_id_and_last_row_by_title(service_sheets, spreadsheet_id)
        # print(data)
        sheet_order_id_발주서, sheet_order_id_발주서매핑모드, sheet_order_id_누적발주, sheet_order_id_상품목록, sheet_order_id_예판상품 = await get_sheet_id_by_title(service_sheets, spreadsheet_id)
        
        # print(sheet_order_id_발주서, sheet_order_id_발주서매핑모드, sheet_order_id_누적발주, sheet_order_id_상품목록, sheet_order_id_예판상품)
        sheet_order_id_발주서, sheet_order_id_발주서매핑모드, sheet_order_id_누적발주, sheet_order_id_상품목록, sheet_order_id_예판상품 = await get_sheet_id_by_title(service_sheets, spreadsheet_id)

        
        protected_range_ids = []
        protected_range_id_발주서 = await get_protected_ranges(service_sheets, spreadsheet_id, sheet_order_id_발주서)
        protected_range_id_매핑모드 = await get_protected_ranges(service_sheets, spreadsheet_id, sheet_order_id_발주서매핑모드)
        protected_range_id_누적발주 = await get_protected_ranges(service_sheets, spreadsheet_id, sheet_order_id_누적발주)
        protected_range_id_상품목록 = await get_protected_ranges(service_sheets, spreadsheet_id, sheet_order_id_상품목록)

        protected_range_ids.extend(protected_range_id_발주서)
        protected_range_ids.extend(protected_range_id_매핑모드)
        protected_range_ids.extend(protected_range_id_누적발주)
        protected_range_ids.extend(protected_range_id_상품목록)

        result_copy_상품목록 = copy_format_to_range("A1:E900", "T1:X900", sheet_order_id_상품목록)
        protected_requests = [create_delete_protected_range_request(pr_id) for pr_id in protected_range_ids]
        requests.extend([
        result_copy_상품목록,
        # create_unlock_all_requests(sheet_order_id_발주서),
        # create_unlock_all_requests(sheet_order_id_발주서매핑모드),
        # create_unlock_all_requests(sheet_order_id_누적발주),
        # create_unlock_all_requests(sheet_order_id_상품목록),
        protected_requests,

        # create_remove_dropdown_request("A8:A8", sheet_order_id_발주서),
        create_clear_values_request("K9:K9", sheet_order_id_발주서),

        # create_remove_dropdown_request("A8:A8", sheet_order_id_발주서매핑모드),
        create_clear_values_request("K9:K9", sheet_order_id_발주서매핑모드),

        create_clear_values_request("A6:E1004", sheet_order_id_상품목록)

        ])

        


        # set_cell_format(cell_range, sheet_order_id, background_color=None, font_size=None, bold=None, border=None, horizontal_alignment="LEFT", vertical_alignment="MIDDLE")

        

        text_일반모드텍스트_A7_발주서 = '현재 시트는 일반모드 발주서입니다.\n매핑모드를 희망하실 경우, 매핑모드 시트로 이동해주세요.'
        formula_매핑모드로이동_A8_발주서 = f'=HYPERLINK("https://docs.google.com/spreadsheets/d/{spreadsheet_id}/edit#gid={sheet_order_id_발주서매핑모드}", "매핑모드바로가기 ←클릭")'
        
        result_일반모드텍스트_A7_발주서 = apply_formula_to_text(text_일반모드텍스트_A7_발주서, "A7:A7", sheet_order_id_발주서)
        result_일반모드포맷_A7_발주서 = set_cell_format("A7:A7", sheet_order_id_발주서, background_color=None, font_size=None, bold=True, border=None, horizontal_alignment="LEFT", vertical_alignment="MIDDLE")
        result_매핑모드로이동_A8_발주서 = apply_formula_to_function(formula_매핑모드로이동_A8_발주서, "A9:A9", sheet_order_id_발주서)
        result_일반모드포맷_A8_발주서 = set_cell_format("A9:A9", sheet_order_id_발주서, background_color=None, font_size=None, bold=True, border=None, horizontal_alignment="LEFT", vertical_alignment="MIDDLE")
        # result_드롭다운A8_발주서 = apply_single_value_dropdown("일반모드", "A8:A8", sheet_order_id_발주서)

        text_매핑모드텍스트_A7_매핑모드 = '현재 시트는 매핑모드 발주서입니다.\n일반모드를 희망하실 경우, 일반모드 시트로 이동해주세요.'
        formula_일반모드로이동_A8_매핑모드 = f'=HYPERLINK("https://docs.google.com/spreadsheets/d/{spreadsheet_id}/edit#gid={sheet_order_id_발주서}", "일반모드바로가기 ←클릭")'

        result_매핑모드텍스트_A7_매핑모드 = apply_formula_to_text(text_매핑모드텍스트_A7_매핑모드, "A7:A7", sheet_order_id_발주서매핑모드)
        result_매핑모드포맷_A7_매핑모드 = set_cell_format("A7:A7", sheet_order_id_발주서매핑모드, background_color=None, font_size=None, bold=True, border=None, horizontal_alignment="LEFT", vertical_alignment="MIDDLE")
        result_일반모드이동_A8_매핑모드 = apply_formula_to_function(formula_일반모드로이동_A8_매핑모드, "A9:A9", sheet_order_id_발주서매핑모드)
        result_일반모드포맷_A8_매핑모드 = set_cell_format("A9:A9", sheet_order_id_발주서매핑모드, background_color=None, font_size=None, bold=True, border=None, horizontal_alignment="LEFT", vertical_alignment="MIDDLE")
        # result_드롭다운A8_매핑모드 = apply_single_value_dropdown("매핑모드", "A8:A8", sheet_order_id_발주서매핑모드)


        text_누적발주이동_J9 = '발주대기 상품은 입금 후 발주 버튼을 체크하면 발주 확정됩니다.\n발주확정 상품은 "누적발주"시트에서 확인 가능합니다.'
        formula_누적발주로이동_M9 = f'=HYPERLINK("https://docs.google.com/spreadsheets/d/{spreadsheet_id}/edit#gid={sheet_order_id_누적발주}", "누적발주 시트 바로가기 ←클릭")'

        result_누적발주이동_J9_발주서 = apply_formula_to_text(text_누적발주이동_J9, "J9:J9", sheet_order_id_발주서)
        result_누적발주이동포맷_J9_발주서 = set_cell_format("J9:J9", sheet_order_id_발주서, background_color=None, font_size=None, bold=True, border=None, horizontal_alignment="LEFT", vertical_alignment="MIDDLE")
        result_누적발주로이동_M9_발주서 = apply_formula_to_function(formula_누적발주로이동_M9, "M9:M9", sheet_order_id_발주서)
        result_누적발주로이동포맷_M9_발주서 = set_cell_format("M9:M9", sheet_order_id_발주서, background_color=None, font_size=None, bold=True, border=None, horizontal_alignment="LEFT", vertical_alignment="MIDDLE")

        result_누적발주이동_J9_매핑모드 = apply_formula_to_text(text_누적발주이동_J9, "J9:J9", sheet_order_id_발주서매핑모드)
        result_누적발주이동포맷_J9_매핑모드 = set_cell_format("J9:J9", sheet_order_id_발주서매핑모드, background_color=None, font_size=None, bold=True, border=None, horizontal_alignment="LEFT", vertical_alignment="MIDDLE")
        result_누적발주로이동_M9_매핑모드 = apply_formula_to_function(formula_누적발주로이동_M9, "M9:M9", sheet_order_id_발주서매핑모드)
        result_누적발주로이동포맷_M9_매핑모드 = set_cell_format("M9:M9", sheet_order_id_발주서매핑모드, background_color=None, font_size=None, bold=True, border=None, horizontal_alignment="LEFT", vertical_alignment="MIDDLE")


        text_배송현황조회_A1_누적발주 = "배송현황조회"
        result_배송현황조회텍스트_A1_누적발주 = apply_formula_to_text(text_배송현황조회_A1_누적발주, "A1:A1", sheet_order_id_누적발주)
        result_배송현황조회포맷_A1_누적발주 = set_cell_format("A1:A1", sheet_order_id_누적발주, background_color=None, font_size=None, bold=True, border=None, horizontal_alignment="LEFT", vertical_alignment="MIDDLE")
        result_체크박스_A2_체크박스 = add_checkbox_to_cell("A2:A2", sheet_order_id_누적발주)




        result_잠금_발주서 = lock_cells_with_warning("J4:T4", sheet_order_id_발주서)
        result_잠금_매핑모드 = lock_cells_with_warning("J4:T4", sheet_order_id_발주서매핑모드)
        result_잠금_누적발주 = lock_cells_with_warning("A4:AF900", sheet_order_id_누적발주)



        formula_A_G열참조_상품목록 = f'=IMPORTRANGE("{spreadsheet_id}", "상품목록!T6:X2000")'
        result_A_G열참조_상품목록 = apply_formula_to_function(formula_A_G열참조_상품목록, "A6:A6", sheet_order_id_상품목록)
        result_상품목록삭제하기_상품목록 = create_remove_dropdown_request("C6:C1004", sheet_order_id_상품목록)

        result_상품목록드롭다운_상품목록 = apply_dropdown_validation("\'상품목록\'!$K$6:$K$800", "V6:V941", sheet_order_id_상품목록)

        result_K1_상품목록 = apply_formula_to_text("1. 상품 목록 보기", "K1:K1", sheet_order_id_상품목록)
        result_K1포맷_상품목록 = set_cell_format("K1:K1", sheet_order_id_상품목록, background_color=background_color, font_size=15, bold=True, horizontal_alignment="CENTER", vertical_alignment="MIDDLE")
        result_U1_상품목록 = apply_formula_to_text("2. 매핑하기", "U1:U1", sheet_order_id_상품목록)
        result_U1포맷_상품목록 = set_cell_format("U1:U1", sheet_order_id_상품목록, background_color=background_color, font_size=15, bold=True, horizontal_alignment="CENTER", vertical_alignment="MIDDLE")

        formula_매핑하는법K3_상품목록 = '=HYPERLINK("https://daebongfarmers.notion.site/2-0-3b25155f593e4b42b828d6dce0f24f54?pvs=4", "붙여넣기로 매핑하는법 영상으로 보기(클릭)")'
        result_매핑하는법K3_상품목록 = apply_formula_to_function(formula_매핑하는법K3_상품목록, "K3:K3", sheet_order_id_상품목록)

        formula_매핑하는법U2_상품목록 = '=HYPERLINK("https://daebongfarmers.notion.site/2-0-3b25155f593e4b42b828d6dce0f24f54?pvs=4", "매핑하는 법 영상으로 보기(클릭)")'
        result_매핑하는법U2_상품목록 = apply_formula_to_function(formula_매핑하는법U2_상품목록, "U2:U2", sheet_order_id_상품목록)

        # result_예시T6_상품목록 = apply_formula_to_text("예시)🍯 꿀맛 달망 비세척 꿀고구마", "T6:T6", sheet_order_id_상품목록)
        # result_예시T6_상품목록 = apply_formula_to_text("예시)🍯 꿀맛 달망 비세척 꿀고구마", "T6:T6", sheet_order_id_상품목록)
        



        # result_진짜잠금_상품목록 = "함수다시짜시넣기"

        result_숨기기A_G_상품목록 = create_hide_columns_request("A1:G900",  sheet_order_id_상품목록)
        result_숨기기AA_BC_상품목록 = create_hide_columns_request("AA1:BC900",  sheet_order_id_상품목록)
        result_전체잠금_상품목록 = apply_protect_data(sheet_order_id_상품목록, "A1:G900")

        result_프로덕트가격_상품목록 = apply_formula_product_everyrow("W6:W6", sheet_order_id_상품목록)
        result_프로덕트매핑확인_상품목록 = apply_formula_mapping_check_everyrow("X6:X6", sheet_order_id_상품목록)
        


#         =IF(AND(ISBLANK('상품목록'!A6), ISBLANK('상품목록'!C6), ISBLANK('상품목록'!D6)), 
#     "", 
#     IF(OR(ISBLANK('상품목록'!A6), ISBLANK('상품목록'!C6), ISBLANK('상품목록'!D6)), 
#         "[매핑오류] 상품명 다시 확인해주세요.", 
#         "[정상매핑] 정상처리 되었습니다.")
# )



        

        # 열 너비 설정 요청 생성
        requests_상품목록_열크기 = [
            set_column_width(sheet_order_id_상품목록, 'T', 193),
            set_column_width(sheet_order_id_상품목록, 'U', 100),
            set_column_width(sheet_order_id_상품목록, 'V', 254),
            set_column_width(sheet_order_id_상품목록, 'W', 100),
            set_column_width(sheet_order_id_상품목록, 'X', 202)
        ]

        
        requests.extend([

# result_전체잠금_상품목록,

result_일반모드텍스트_A7_발주서,
result_일반모드포맷_A7_발주서,
result_매핑모드로이동_A8_발주서,
# result_드롭다운A8_발주서,


result_매핑모드텍스트_A7_매핑모드,
result_매핑모드포맷_A7_매핑모드,
result_일반모드이동_A8_매핑모드,
# result_드롭다운A8_매핑모드,

result_누적발주이동_J9_발주서,
result_누적발주이동포맷_J9_발주서,
result_누적발주로이동_M9_발주서,
result_누적발주로이동포맷_M9_발주서,

result_누적발주이동_J9_매핑모드,
result_누적발주이동포맷_J9_매핑모드,
result_누적발주로이동_M9_매핑모드,
result_누적발주로이동포맷_M9_매핑모드,

result_배송현황조회텍스트_A1_누적발주,
result_배송현황조회포맷_A1_누적발주,
result_체크박스_A2_체크박스,

result_잠금_발주서,
result_잠금_매핑모드,
result_잠금_누적발주,


result_일반모드포맷_A8_매핑모드,
result_일반모드포맷_A8_발주서,


result_A_G열참조_상품목록,
result_상품목록삭제하기_상품목록,
            
result_숨기기A_G_상품목록,
result_숨기기AA_BC_상품목록,
requests_상품목록_열크기,
result_상품목록드롭다운_상품목록,
result_전체잠금_상품목록,

result_K1_상품목록,
result_U1_상품목록,
result_매핑하는법K3_상품목록,
result_매핑하는법U2_상품목록,

result_K1포맷_상품목록,
result_U1포맷_상품목록
        ])

        requests.extend(result_프로덕트가격_상품목록)
        requests.extend(result_프로덕트매핑확인_상품목록)
        
        if len(requests) > 0:
            await batchWriteRequest(service_sheets, spreadsheet_id, requests)
        




        



        # formula_공급금액_발주서 = f'=IF(SUM(J$12:J) < 0, 0, SUM(J$12:J))'
        # result_공급금액_발주서 = apply_formula_to_function(formula_공급금액_발주서, "L4:L4", sheet_order_id_발주서)
        # result_공급금액_발주서매핑모드 = apply_formula_to_function(formula_공급금액_발주서, "L4:L4", sheet_order_id_발주서매핑모드)

        # result_쿼리_예판상품 = apply_formula_to_function(query_formula_예판상품, "B6:B6", sheet_order_id_예판상품)
        # requests.append(result_공급금액_발주서)
        # requests.append(result_공급금액_발주서매핑모드)
        # requests.append(result_쿼리_예판상품)

        # if len(requests) > 0:
        #     await batchWriteRequest(service_sheets, spreadsheet_id, requests)



user_data_list
[['', '407', '윤경', '407_윤경', '', '', '', '', '', '1057789072', '16Enb8YKihLDGX71hvbBX8KtJ7H-Mco-WQukPzW101zo'], ['', '408', '장병식', '408_장병식', '', '', '', '', '', '1027140764', '1wgnpQR7TPJmMPh9RUfmrPiRrcOWKgqtFdkPlqFQg5fM'], ['', '409', '이희중', '409_이희중', '', '', '', '', '', '1049307101', '1KcK5g3IyPTrE3LfiG4MDROCGevrq7u23R-LZLaF34No'], ['', '410', '정은아', '410_정은아', '', '', '', '', '', '1080801353', '1vrB00dzNDDXiMrPNt93fU9NZRUcbhEWRUSSih6ZCgzs'], ['', '411', '남진주 ', '411_남진주 ', '', '', '', '', '', '1071899088', '14NcftN5VzDLrpkAsNKUqErfYxWtE3fyo-qUKC3Pocak'], ['', '412', '김수연', '412_김수연', '', '', '', '', '', '1077705213', '1aVnVQJGtATIHJlW-MIL-w3VhnaJFzwImlX9ostyHoRA'], ['', '413', '박영미 ', '413_박영미 ', '', '', '', '', '', '1082105445', '1ZQSrieMm8LzpQUJ_nXceEkFQSG1fntbH-6tz3ofzyUE'], ['', '414', '박영미 ', '414_박영미 ', '', '', '', '', '', '1082105445', '1hLpxyUACaR5NsUxnz0eo_pcsb1FKnC4C6Mi9Dor5yWc'], ['', '415', '조성민', '415_조성민', '', '', '', '', '', '1083479767', '18eG80C0I8F

In [427]:
# def get_format_from_range(spreadsheet_id, sheet_id, range_name, service):

#     range_value = convert_range_to_indices(sheet_id, range_name)

#     request = {
#         "ranges": [range_name],
#         "includeGridData": True
#     }

#     result = service.spreadsheets().get(spreadsheetId=spreadsheet_id, ranges=request["ranges"], includeGridData=request["includeGridData"]).execute()
    
#     # 서식 정보 추출
#     sheet_data = result.get('sheets', [])[0]
#     row_data = sheet_data.get('data', [])[0].get('rowData', [])
#     cell_format_data = []

#     print(row_data)

#     for row in row_data:
#         cell_data = row.get('values', [])[0]  # 한 셀만 가정
#         format_info = cell_data.get('userEnteredFormat', {})
#         cell_format_data.append(format_info)
    
#     return cell_format_data

# def get_format_from_range(spreadsheet_id, sheet_id, range_name, service):
#     request = {
#         "ranges": [range_name],
#         "includeGridData": True
#     }

#     result = service.spreadsheets().get(spreadsheetId=spreadsheet_id, ranges=request["ranges"], includeGridData=request["includeGridData"]).execute()
    
#     # 서식 정보 추출
#     sheet_data = result.get('sheets', [])[0]
#     row_data = sheet_data.get('data', [])[0].get('rowData', [])
#     cell_format_data = []

#     for row in row_data:
#         row_format = []
#         for cell in row.get('values', []):
#             format_info = cell.get('userEnteredFormat', {})
#             row_format.append(format_info)
#         cell_format_data.append(row_format)
    
#     return cell_format_data

# def get_format_from_range(spreadsheet_id, sheet_id, ranges, service):
#     ranges = ["U1:X1", "U2:X2", "V4:V4", "T5:X5", "T6:X6"] 
#     request = {
#         "ranges": ranges,
#         "includeGridData": True
#     }

#     result = service.spreadsheets().get(spreadsheetId=spreadsheet_id, ranges=request["ranges"], includeGridData=request["includeGridData"]).execute()
    
#     cell_format_data = []

#     for sheet in result.get('sheets', []):
#         for data in sheet.get('data', []):
#             row_data = data.get('rowData', [])
#             for row in row_data:
#                 row_format = []
#                 for cell in row.get('values', []):
#                     format_info = cell.get('userEnteredFormat', {})
#                     row_format.append(format_info)
#                 cell_format_data.append(row_format)
    
#     return cell_format_data

# def get_format_from_range(spreadsheet_id, sheet_id, ranges, service):
#     requests = []
#     for range_name in ranges:
#         requests.append({
#             "ranges": [range_name],
#             "includeGridData": True
#         })

#     result = service.spreadsheets().get(spreadsheetId=spreadsheet_id, ranges=[r["ranges"][0] for r in requests], includeGridData=True).execute()
    
#     cell_format_data = []

#     for sheet in result.get('sheets', []):
#         for data in sheet.get('data', []):
#             row_data = data.get('rowData', [])
#             for row in row_data:
#                 row_format = []
#                 for cell in row.get('values', []):
#                     format_info = cell.get('userEnteredFormat', {})
#                     row_format.append(format_info)
#                 cell_format_data.append(row_format)
    
#     return cell_format_data

# # 이미 있는 코드에 포함된 함수들
# def convert_range_to_indices(sheet_id, cell_range):
#     """
#     주어진 셀 범위를 rowIndex와 columnIndex로 변환합니다.
    
#     :param sheet_id: 변환할 시트의 ID
#     :param cell_range: 변환할 셀 범위 (예: "A2:B10")
#     :return: 변환된 범위 딕셔너리
#     """
#     import re

#     # A1 표기법을 정규 표현식으로 분리
#     match = re.match(r"([A-Z]+)([0-9]+):([A-Z]+)([0-9]+)", cell_range)
#     if not match:
#         raise ValueError("셀 범위 형식이 올바르지 않습니다. 예: 'A2:B10'")
    
#     start_col, start_row, end_col, end_row = match.groups()
    
#     start_row_index = int(start_row) - 1
#     end_row_index = int(end_row)
#     start_col_index = col_to_index(start_col)
#     end_col_index = col_to_index(end_col) + 1
    
#     return {
#         'sheetId': sheet_id,
#         'startRowIndex': start_row_index,
#         'endRowIndex': end_row_index,
#         'startColumnIndex': start_col_index,
#         'endColumnIndex': end_col_index
#     }

# def col_to_index(col):
#     index = 0
#     for c in col:
#         index = index * 26 + (ord(c) - ord('A') + 1)
#     return index - 1

# # 특정 셀 범위의 서식을 가져오는 예제
# # def fetch_format(spreadsheet_id, sheet_id, range_name):
# #     range_name = "A1:A1"  # 서식을 복사할 원본 셀 범위
# #     format_data = get_format_from_range(spreadsheet_id, sheet_id, range_name)
# #     return format_data


# if __name__ == "__main__":
#     creds = oauthByWeb()
#     service_sheets = build('sheets', 'v4', credentials=creds)
#     gridObject = get_format_from_range("1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw", "2067917506", "T1:X10", service_sheets)

#     print(gridObject)

In [428]:
# from googleapiclient.discovery import build

# def get_sheet_id_by_name(service, spreadsheet_id, sheet_name):
#     spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
#     sheets = spreadsheet.get('sheets', [])
#     for sheet in sheets:
#         if sheet.get("properties", {}).get("title") == sheet_name:
#             return sheet.get("properties", {}).get("sheetId")
#     return None

# def get_format_from_range(spreadsheet_id, ranges, service):
#     result = service.spreadsheets().get(spreadsheetId=spreadsheet_id, ranges=ranges, includeGridData=True).execute()
    
#     cell_format_data = []

#     for sheet in result.get('sheets', []):
#         for data in sheet.get('data', []):
#             row_data = data.get('rowData', [])
#             for row in row_data:
#                 row_format = []
#                 for cell in row.get('values', []):
#                     format_info = cell.get('userEnteredFormat', {})
#                     row_format.append(format_info)
#                 cell_format_data.append(row_format)
    
#     return cell_format_data

# # 이미 있는 코드에 포함된 함수들
# def convert_range_to_indices(sheet_id, cell_range):
#     """
#     주어진 셀 범위를 rowIndex와 columnIndex로 변환합니다.
    
#     :param sheet_id: 변환할 시트의 ID
#     :param cell_range: 변환할 셀 범위 (예: "A2:B10")
#     :return: 변환된 범위 딕셔너리
#     """
#     import re

#     # A1 표기법을 정규 표현식으로 분리
#     match = re.match(r"([A-Z]+)([0-9]+):([A-Z]+)([0-9]+)", cell_range)
#     if not match:
#         raise ValueError("셀 범위 형식이 올바르지 않습니다. 예: 'A2:B10'")
    
#     start_col, start_row, end_col, end_row = match.groups()
    
#     start_row_index = int(start_row) - 1
#     end_row_index = int(end_row)
#     start_col_index = col_to_index(start_col)
#     end_col_index = col_to_index(end_col) + 1
    
#     return {
#         'sheetId': sheet_id,
#         'startRowIndex': start_row_index,
#         'endRowIndex': end_row_index,
#         'startColumnIndex': start_col_index,
#         'endColumnIndex': end_col_index
#     }

# def col_to_index(col):
#     index = 0
#     for c in col:
#         index = index * 26 + (ord(c) - ord('A') + 1)
#     return index - 1

# # 특정 셀 범위의 서식을 가져오는 예제
# def fetch_format(spreadsheet_id, sheet_name, service):
#     sheet_id = get_sheet_id_by_name(service, spreadsheet_id, sheet_name)
#     if sheet_id is None:
#         raise ValueError(f"Sheet with name '{sheet_name}' not found.")
    
#     ranges = [f"{sheet_name}!U1:X1", f"{sheet_name}!U2:X2", f"{sheet_name}!V4:V4", f"{sheet_name}!T5:X5", f"{sheet_name}!T6:X6"]  # 서식을 복사할 원본 셀 범위
#     format_data = get_format_from_range(spreadsheet_id, ranges, service)
#     return format_data



# if __name__ == "__main__":
#     creds = oauthByWeb()
#     service_sheets = build('sheets', 'v4', credentials=creds)
#     gridObject = fetch_format("1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw", "상품목록", service_sheets)
#     print(gridObject)


In [429]:
print(col_to_index("W"))

22
